In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.externals import joblib
from cv2 import HOGDescriptor

def compute_features(image):
    winSize = (16,48)
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 100
    nlevels = 32
    hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    winStride = (8,8)
    padding = (16,16)
    hist = hog.compute(image,winStride,padding)
    return np.reshape(hist, 4500)


clf = joblib.load("model_semafors.pkl") 

cap = cv2.VideoCapture("test2.mp4")
frame = cap.read()[1]
frame = cv2.resize(frame,(640, 480), interpolation = cv2.INTER_CUBIC)
height, width = frame.shape[:2]
while(cap.isOpened()):
    frame=cap.read()[1]
    frame = cv2.resize(frame,(640, 480), interpolation = cv2.INTER_CUBIC)
    frame = cv2.GaussianBlur(frame,(3,3),0)
    r,g,b = cv2.split(frame)
    circles = cv2.HoughCircles(b,cv2.HOUGH_GRADIENT,0.5,50,param1=70,param2=10,minRadius=2,maxRadius=20)
    if np.all(circles) != None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
#            print(frame[i[1]][i[0]])
            if frame[i[1]][i[0]][2] > 160 and frame[i[1]][i[0]][1]<150 and frame[i[1]][i[0]][0]<150:
#                print((i[0]-i[2]*2,i[1]-i[2]*2),(i[0]+i[2]*2,i[1]+i[2]*7))
                x1=i[0]-i[2]*2
                y1=i[1]-i[2]*2
                x2=i[0]+i[2]*2
                y2=i[1]+i[2]*7
                if y2>479:
                    y2=479
                if x2>639:
                    x2=639
                if(x1 != 0 and x2 !=0 and y1!=0 and y2!=0):
                    cut_frame = frame[y1:y2,x1:x2]
                    try:
                        cut_frame = cv2.resize(cut_frame,(16, 48), interpolation = cv2.INTER_CUBIC)
                        pred = clf.predict_proba([compute_features(cut_frame)])[0][1]
#                       cv2.imshow('frame '+str(pred),cut_frame)
#                       cv2.circle(frame,(i[0],i[1]),i[2],(0,255,255),2)
#                       frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                        if(pred>0.7):                           
                            cv2.circle(frame,(i[0],i[1]),i[2],(0,255,255),2)
                            frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                            
                    except:
                        print("error")
    cv2.imshow('frame2',frame)
    if cv2.waitKey(2) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()